### MCP(Model Context Protocol) 模型上下文协议

##### 痛点
- 语言模型知识数据库有限
- 早期语言模型能力不够强大，幻觉严重
##### 需求
- 让语言模型可以获取到最新的数据
- 基于语言模型更多权限，能够与聊天以外的工具进行交互

#### Tool Use

>- The agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.
>- 智能体学习调用外部 API 以获取模型权重中缺失的额外信息（通常在预训练后难以更改），包括当前信息、代码执行能力、对专有信息源的访问等。

#### 提示词工程
通过提示词工程，定义协议，可以让用户按照协议输入，也可以让大模型返回协议，解析文本中的返回，之后**交给工程代码去调用**。

##### 流程
1. 定义协议，让LLM遵循
2. 用户发起发起查询
3. LLM处理用户查询，通过自然语言理解用户的意图，可能会请求获取更多信息或者使用某个工具
4. 用户解析LLM输出，通过匹配规则去执行相应的工具
5. 用户返回工具所需参数
6. LLM格式化工具调用，响应请求参数
7. 用户本地执行工具，返回工具响应结果
8. LLM观测数据，并回答结果

<br>

```text
     +-----------------+                               +-------------------+
     |                 |--(A)- 意图澄清/信息请求 -------->|        用户        |
     |   LLM / 代理     |                               |                   |
     | (基于Few-Shot    |<-(B)-- 查询详述/用户确认 --------|                   |
     |   规范决策)       |                               +-------------------+
     +-----------------+
           |
           |  (LLM 内部处理:
           |   根据 (B) 和内置的 Few-Shot 规范,
           |   进行工具选择和调用格式化)
           |
     +-----------------+                               +-------------------+
     |                 |--(C)-- 格式化工具调用 --------->|   外部工具 (API)    |
     |   LLM / 代理     |                              |                    |
     |                 |<-(D)--- 工具结果/观测数据 ------|                    |
     +-----------------+                               +-------------------+
```

##### 痛点
- 早期大模型能力不足，难以严格遵守指令
- 早期大模型返回内容可能会不标准，可能会多余返回：“好的，下面是您需要的..."类似的开场词
- 早期大模型上下文有限，难以处理复杂参数工具

In [41]:
// Message 结构用于表示聊天消息
type Message struct {
	Role    string `json:"role"`
	Content string `json:"content"`
}

// RequestPayload 结构用于表示发送给 API 的整个请求体
type RequestPayload struct {
	Model    string    `json:"model"`
	Messages []Message `json:"messages"`
}

In [42]:
func sendHTTPRequest(url string, apiKey string, payload RequestPayload) (string, string) {
  jsonData, err := json.Marshal(payload)
  if err != nil {
    log.Fatalf("无法编码 JSON 请求体: %v", err)
  }

  req, err := http.NewRequest("POST", url, bytes.NewBuffer(jsonData))
  if err != nil {
    log.Fatalf("无法创建 HTTP 请求: %v", err)
  }

  req.Header.Set("Content-Type", "application/json")
  req.Header.Set("Authorization", "Bearer "+apiKey)

  client := &http.Client{}
  resp, err := client.Do(req)
  if err != nil {
    log.Fatalf("发送 HTTP 请求失败: %v", err)
  }
  defer resp.Body.Close()

  body, err := ioutil.ReadAll(resp.Body)
  if err != nil {
    log.Fatalf("无法读取 HTTP 响应体: %v", err)
  }

  return string(body), resp.Status
}

In [55]:
var prompts = `
你是一个智能助手，具有以下工具：

1. 天气工具 (WEATHER)
   - 格式：WEATHER: 城市名称
   - 功能：获取指定城市的当前天气情况

2. 计算工具 (CALCULATE)
   - 格式：CALCULATE: 数学表达式
   - 功能：执行数学计算

重要规则：
- 必须严格遵循上述格式
- 只有在确实需要使用工具时才使用
- 工具调用应该是输出的唯一内容
- 使用工具后，还需要对结果进行自然语言解释

示例：

用户: 我想知道北京的天气
助手: WEATHER: 北京

用户: 帮我计算15乘以23
助手: CALCULATE: 15 * 23
`

%%
var (
  url    = "https://api.gptapi.us/v1/chat/completions"
  apiKey = "sk-8tp4stVRMEgVFrCWF54c8eA1265c47B384618c96A11c790c"
)

// 构建请求体数据
payload := RequestPayload{
  Model: "gpt-4o-mini",
  Messages: []Message{
    {
      Role: "developer",
      Content: prompts,
    },
    {
      Role:    "user",
      Content: "请告诉我北京的天气",
    },
  },
}

body, resp := sendHTTPRequest(url, apiKey, payload)

fmt.Println("响应状态码:", resp)
var prettyJSON bytes.Buffer
if err := json.Indent(&prettyJSON, []byte(body), "", "  "); err == nil {
  fmt.Println("响应体:\n", prettyJSON.String())
} else {
  fmt.Println("响应体 (原始):\n", string(body))
}

响应状态码: 200 OK
响应体:
 {
  "id": "chatcmpl-9JatsKPgRZ0oUstvZCt5Mw7B2HdTb",
  "object": "chat.completion",
  "created": 1747243009,
  "model": "gpt-4o-mini",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "WEATHER: \u5317\u4eac\n",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "system_fingerprint": "fp_958fe43bb7",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 0,
    "total_tokens": 2,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


##### Function Calling

> 你可以通过函数调用让模型访问你自己的自定义代码。根据系统提示和消息，模型可能会决定调用这些函数——而不是（或除了）生成文本或音频。
> 然后，你将执行函数代码，发回结果，模型会将这些结果整合到其最终响应中。

<img src="pics/function-calling-diagram-steps.png" width="40%">